# 线性回归的简洁实现

在本节中，我们将介绍如何(**通过使用深度学习框架来简洁地实现**)(**线性回归模型**)。

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

## 生成数据集

我们首先[**生成数据集**]。

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

我们可以[**调用框架中现有的API来读取数据**]。
我们将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [6]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    
    # data.TensorDataset包装数据和目标张量的数据集。
    dataset = data.TensorDataset(*data_arrays)
    
    # data.DataLoader 数据加载器。组合数据集和采样器，并在数据集上提供单进程或多进程迭代器。
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [7]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

这里我们使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。

In [8]:
next(iter(data_iter))

[tensor([[ 0.8771, -1.2407],
         [-1.3897, -0.2720],
         [-0.7329,  1.1105],
         [-0.5953, -0.1897],
         [-0.4715,  0.2318],
         [-0.4756,  1.1297],
         [-0.3678, -0.8144],
         [ 1.2509,  1.2487],
         [ 0.5960, -0.7754],
         [ 1.4840, -0.4353]]),
 tensor([[10.1792],
         [ 2.3516],
         [-1.0277],
         [ 3.6443],
         [ 2.4804],
         [-0.6005],
         [ 6.2369],
         [ 2.4454],
         [ 8.0226],
         [ 8.6417]])]

## 定义模型

对于标准深度学习模型，我们可以[**使用框架的预定义好的层**]。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
我们首先定义一个模型变量`net`，它是一个`Sequential`类的实例。
`Sequential`类将多个层串联在一起。
当给定输入数据时，`Sequential`实例将数据传入到第一层，
然后将第一层的输出作为第二层的输入，以此类推。
在下面的例子中，我们的模型只包含一个层，因此实际上不需要`Sequential`。
但是由于以后几乎所有的模型都是多层的，在这里使用`Sequential`会让你熟悉“标准的流水线”。

在PyTorch中，全连接层在`Linear`类中定义。
值得注意的是，我们将两个参数传递到`nn.Linear`中。
第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [10]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## (**初始化模型参数**)

在使用`net`之前，我们需要初始化模型参数。
如在线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化参数。
在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，
偏置参数将初始化为零。

正如我们在构造`nn.Linear`时指定输入和输出尺寸一样，
现在我们能直接访问参数以设定它们的初始值。
我们通过`net[0]`选择网络中的第一个图层，
然后使用`weight.data`和`bias.data`方法访问参数。
我们还可以使用替换方法`normal_`和`fill_`来重写参数值。

In [15]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

[**计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数**]。
默认情况下，它返回所有样本损失的平均值。

In [16]:
loss = nn.MSELoss()

## 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具，
PyTorch在`optim`模块中实现了该算法的许多变种。
当我们(**实例化一个`SGD`实例**)时，我们要指定优化的参数
（可通过`net.parameters()`从我们的模型中获得）以及优化算法所需的超参数字典。
小批量随机梯度下降只需要设置`lr`值，这里设置为0.03。

In [18]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

In [20]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000222
epoch 2, loss 0.000101
epoch 3, loss 0.000100
